In [ ]:
#--------------------
# Necessary Libraries
#--------------------
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, SubsetRandomSampler

In [ ]:
#----------------------
# Log in to W&B account
#----------------------
import wandb
wandb.login(key='150002a34bcf7d04848ccaff65ab76ca5cc3f11b')

In [ ]:
#-------------------------
# Inaturalist-dataset path
#-------------------------
data_dir = "/kaggle/input/inaturalist-dataset/nature_12K/inaturalist_12K"

In [ ]:
#---------------------------
# ResNet50 Finetune Function
#---------------------------
def load_resnet50_finetune_model(
    mode="head",                 # options: "head", "partial", "full"
    num_classes=10,
    dropout_rate=0.0,
    unfreeze_from="layer3"       # if mode == "partial"
):
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

    # Replace the final fully connected layer
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(in_features, num_classes)
    )

    if mode == "head":
        # Freeze all layers except the final classification head
        for name, param in model.named_parameters():
            if not name.startswith("fc"):
                param.requires_grad = False

    elif mode == "partial":
        # Partially unfreeze the model, training layers from 'unfreeze_from' onwards
        freeze = True
        for name, module in model.named_children():
            if name == unfreeze_from:
                freeze = False
            for param in module.parameters():
                param.requires_grad = not freeze

    elif mode == "full":
        # Unfreeze the entire model, training all layers
        for param in model.parameters():
            param.requires_grad = True

    else:
        raise ValueError("Mode must be one of: 'head', 'partial', or 'full'")

    return model

In [ ]:
#-------------------------------
# Data Loading and Preprocessing
#-------------------------------
def load_and_preprocess_data(data_dir, batch_size, validation_split=0.2):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),                   # Resize images
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])   # Standardize the pixel values
    ])

    train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
    test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=transform)

    # Training and validation splits
    dataset_size = len(train_dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)

    return train_loader, val_loader, test_dataset

In [ ]:
#-------------------
# Training the model
#-------------------
def train(model, train_loader, val_loader, optimizer, criterion, epochs, device):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Train loss and Accuracy
            train_loss += loss.item() * labels.size(0)  # Accumulate loss
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
        avg_train_loss = train_loss / total
        train_accuracy = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
        avg_val_loss = val_loss / val_total
        val_accuracy = val_correct / val_total

        # Log in wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": avg_val_loss,
            "val_accuracy": val_accuracy
        })
        print(f"Epoch [{epoch+1}/{epochs}], Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
#--------------------------
# Wandb Sweep Configuration
#--------------------------
sweep_config = {
    "method": "bayes",  # Bayesian optimization for efficiency
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'epochs' : {'values':[5,10]},
        #'mode': {'values': ['head', 'partial', 'full']},
        'dropout': {'values': [0.0, 0.2, 0.3]},
        #'unfreeze_from': {'values': ['layer2', 'layer3', 'layer4']},
        'batch_size': {'values': [32, 64]},
        'learning_rate': {'values': [1e-3, 1e-4]}
    }
}

In [ ]:
#---------------------
# Wandb Sweep Function
#---------------------
def wandb_sweep():
    
    wandb.init(project="iNaturalist-ResNet50")
    # Access sweep configuration from wandb
    config = wandb.config

    # Run name
    run_name = f"ep-{config.epochs}_dro-{config.dropout}_bs-{config.batch_size}_lr-{config.learning_rate}"
    wandb.run.name = run_name

    # Data Loading
    data_dir = "/kaggle/input/inaturalist-dataset/nature_12K/inaturalist_12K"
    train_loader, val_loader, test_dataset = load_and_preprocess_data(data_dir, config.batch_size)

    model = load_resnet50_finetune_model(
        mode='head',   #config.mode,
        dropout_rate=config.dropout,
        #unfreeze_from=config.unfreeze_from,
        num_classes=10
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

    # Training
    train(model, train_loader, val_loader, optimizer, criterion, epochs=config.epochs, device=device)

    # Evaluate on test set
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size)
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="iNaturalist-ResNet50")
    wandb.agent(sweep_id, wandb_sweep)